<a href="https://colab.research.google.com/github/khadeejahalam/KHADEEJAH-/blob/main/22i2063_khadeejah_C_BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import zipfile
import io
from math import log

class WordEnumeration:

    @staticmethod
    def extract_words(line):
        words = re.findall(r'\b\w+\b', line.lower())
        for word in words:
            yield word, 1

class DocumentCount:

    @staticmethod
    def count_words(line):
        words = re.findall(r'\b\w+\b', line.lower())
        for word in set(words):
            yield word, 1

class Indexer:

    @staticmethod
    def index_words(line):
        doc_id, text = line.split(',', 1)
        words = re.findall(r'\b\w+\b', text.lower())
        word_counts = {}
        total_words = len(words)
        for word in words:
            word_counts[word] = word_counts.get(word, 0) + 1

        for word, count in word_counts.items():
            yield word, (doc_id, count / total_words)

class QueryProcessor:

    def __init__(self, query):
        self.query_terms = query.lower().split()

    def process_query(self, line):
        doc_id, vector = line.split('\t', 1)
        vector = eval(vector)
        for term, tf in vector:
            if term in self.query_terms:
                yield doc_id, tf

def read_file_from_zip(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            with zip_ref.open(file_name) as file:
                for line in io.TextIOWrapper(file, 'utf-8'):
                    yield line

# Specify the path to the zipped Wikipedia dataset file
input_zip_path = "/content/enwiki-20170820.zip"

# Specify your query here
query = "your_query_here"

# Running Word Enumeration
word_enum = WordEnumeration()
word_counts = {}
for line in read_file_from_zip(input_zip_path):
    for word, count in word_enum.extract_words(line):
        word_counts[word] = word_counts.get(word, 0) + count

# Running Document Count
doc_counts = {}
doc_count = DocumentCount()
for line in read_file_from_zip(input_zip_path):
    for word, count in doc_count.count_words(line):
        doc_counts[word] = doc_counts.get(word, 0) + count

# Running Indexer
indexed_words = {}
indexer = Indexer()
for line in read_file_from_zip(input_zip_path):
    for word, index in indexer.index_words(line):
        if word not in indexed_words:
            indexed_words[word] = []
        indexed_words[word].append(index)

# Running Query Processor
query_processor = QueryProcessor(query)
query_results = {}
for line in read_file_from_zip(input_zip_path):
    for doc_id, tf in query_processor.process_query(line):
        if doc_id not in query_results:
            query_results[doc_id] = 0
        query_results[doc_id] += tf
